In [81]:
import numpy as np
import shutil
import os
from ultralytics import YOLO
import cv2
from tkinter import messagebox
import time
import PySide2

In [82]:
model = YOLO("Weights\\Small (n)\\Trial 3\\best.pt")

model.predict(r"C:\Users\GillA\Desktop\University\\demo3.mp4",save_txt=True)


Ultralytics YOLOv8.0.40  Python-3.9.16 torch-1.13.1+cpu CPU
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs

video 1/1 (1/911) C:\Users\GillA\Desktop\University\Demo3.mp4: 384x640 1 LeBron, 253.0ms
video 1/1 (2/911) C:\Users\GillA\Desktop\University\Demo3.mp4: 384x640 1 LeBron, 207.4ms
video 1/1 (3/911) C:\Users\GillA\Desktop\University\Demo3.mp4: 384x640 1 LeBron, 234.3ms
video 1/1 (4/911) C:\Users\GillA\Desktop\University\Demo3.mp4: 384x640 1 LeBron, 269.8ms
video 1/1 (5/911) C:\Users\GillA\Desktop\University\Demo3.mp4: 384x640 1 LeBron, 362.8ms
video 1/1 (6/911) C:\Users\GillA\Desktop\University\Demo3.mp4: 384x640 1 LeBron, 335.6ms
video 1/1 (7/911) C:\Users\GillA\Desktop\University\Demo3.mp4: 384x640 1 LeBron, 312.7ms
video 1/1 (8/911) C:\Users\GillA\Desktop\University\Demo3.mp4: 384x640 1 LeBron, 257.2ms
video 1/1 (9/911) C:\Users\GillA\Desktop\University\Demo3.mp4: 384x640 1 LeBron, 212.6ms
video 1/1 (10/911) C:\Users\GillA\Desktop\University\Demo3.

[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
 type: <class 'torch.Tensor'>
 shape: torch.Size([1, 6])
 dtype: torch.float32
  + tensor([[8.10000e+02, 1.98000e+02, 1.16100e+03, 9.09000e+02, 8.81304e-01, 0.00000e+00]]),
 Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
 type: <class 'torch.Tensor'>
 shape: torch.Size([1, 6])
 dtype: torch.float32
  + tensor([[8.03000e+02, 1.98000e+02, 1.18300e+03, 9.06000e+02, 9.02535e-01, 0.00000e+00]]),
 Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
 type: <class 'torch.Tensor'>
 shape: torch.Size([1, 6])
 dtype: torch.float32
  + tensor([[8.03000e+02, 1.98000e+02, 1.18300e+03, 9.05000e+02, 9.02910e-01, 0.00000e+00]]),
 Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
 type: <class 'torch.Tensor'>
 shape: torch.Size([1, 6])
 dtype: torch.float32
  + tensor([[7.85000e+02, 1.98000e+02, 1.18600e+03, 9.11000e+02, 9.11795e-01, 0.00000e+00]]),
 Ultralytics YOLO <c

In [83]:
def yolo_text_to_dict(lines, classes):
    
    dic = {}
        
    items = {}
        
    for i in lines:
        i.strip()
        obj = i.split(" ")[0]
        x_coord = i.split(" ")[1]
        y_coord = i.split(" ")[2]
        width = i.split(" ")[3]
        height = i.split(" ")[4]
        
        
        items["X_coord"] = float(x_coord)
        items["Y_coord"] = float(y_coord)
        items["Width"] = float(width)
        items["Height"] = float(height)
        
        dic[classes[int(obj)]] = dict(items)
        
        
    return dic

In [84]:
video_num = len(list(os.listdir("Video")))
# video_num = 0

classes = ["LeBron", "Ball", "Net"]

video_dir = "Video/video_" + str(video_num + 1)
annot_dir = "Video/video_" + str(video_num + 1) + "/annots" 

if not os.path.exists(video_dir):
    os.mkdir(video_dir)
    
if not os.path.exists(annot_dir):
    os.mkdir(annot_dir)

if os.path.exists("runs\detect"):
    for i in list(os.listdir("runs\detect\predict\labels")):


        shutil.move("runs\detect\predict\labels\\" + i, annot_dir)
    
    shutil.rmtree("runs\detect")
                  

frame_max = [] 

for i in list(os.listdir(annot_dir)):
        
    frame_max.append(int(i.split(".")[0].split("_")[1]))
    
frame_max = max(frame_max)

frames_info = {}

for i in range(1,frame_max+1):
        
    path = annot_dir + "\\demo3_" + str(i) + ".txt"
    
    if os.path.exists(path):
    
        with open(path) as f:

            lines = f.readlines()

        frames_info["Frame_" + str(i)] = yolo_text_to_dict(lines, classes)
        
    else:
        frames_info["Frame_" + str(i)] = {}


if len(list(os.listdir(annot_dir))) == 0:
    shutil.rmtree(video_dir)

In [85]:
frames_info

{'Frame_1': {'LeBron': {'X_coord': 0.513281,
   'Y_coord': 0.5125,
   'Width': 0.182812,
   'Height': 0.658333}},
 'Frame_2': {'LeBron': {'X_coord': 0.517187,
   'Y_coord': 0.511111,
   'Width': 0.197917,
   'Height': 0.655556}},
 'Frame_3': {'LeBron': {'X_coord': 0.517187,
   'Y_coord': 0.510648,
   'Width': 0.197917,
   'Height': 0.65463}},
 'Frame_4': {'LeBron': {'X_coord': 0.513281,
   'Y_coord': 0.513426,
   'Width': 0.208854,
   'Height': 0.660185}},
 'Frame_5': {'LeBron': {'X_coord': 0.510156,
   'Y_coord': 0.522685,
   'Width': 0.222396,
   'Height': 0.656482}},
 'Frame_6': {'LeBron': {'X_coord': 0.510156,
   'Y_coord': 0.523148,
   'Width': 0.222396,
   'Height': 0.657407}},
 'Frame_7': {'LeBron': {'X_coord': 0.503906,
   'Y_coord': 0.526852,
   'Width': 0.220313,
   'Height': 0.655556}},
 'Frame_8': {'LeBron': {'X_coord': 0.503906,
   'Y_coord': 0.526852,
   'Width': 0.220313,
   'Height': 0.655556}},
 'Frame_9': {'LeBron': {'X_coord': 0.50625,
   'Y_coord': 0.53287,
   'Widt

In [90]:
def shot_check(curr_frame, waiting):
    
    if "Ball" in list(curr_frame.keys()) and "Net" in list(curr_frame.keys()) and waiting==0:

        ball = curr_frame["Ball"]
        net = curr_frame["Net"]
        
        ball_above_net = ball["Y_coord"] < (net["Y_coord"])
        
        ball_aligned_r = (ball["X_coord"] > (net["X_coord"] - 2*ball["Width"]))
        ball_aligned_l = (ball["X_coord"] < (net["X_coord"] + 2*ball["Width"]))        
        
        if  ball_above_net and ball_aligned_r and ball_aligned_l:
            return("Shot Up")
        
    elif "Ball" in list(curr_frame.keys()) and "Net" in list(curr_frame.keys()) and waiting > 0:
        
        ball = curr_frame["Ball"]
        net = curr_frame["Net"]
        
        ball_below_net = ball["Y_coord"] > (net["Y_coord"])
        
        ball_aligned_r = (ball["X_coord"] > (net["X_coord"] - 0.5*ball["Width"]))
        ball_aligned_l = (ball["X_coord"] < (net["X_coord"] + 0.5*ball["Width"]))        
        
        if  ball_below_net and ball_aligned_r and ball_aligned_l:
            return("+1 FG")
        
    return 0

In [91]:
x = (frames_info["Frame_1"])

def is_lebron_with_ball(prev_count, curr_frame):

    if "Ball" in list(curr_frame.keys()) and "LeBron" in list(curr_frame.keys()):


        if prev_count > 45:
            prev_count = 0


        LeBron_X = curr_frame["LeBron"]["X_coord"]
        LeBron_Y = curr_frame["LeBron"]["Y_coord"]
        Ball_X = curr_frame["Ball"]["X_coord"]
        Ball_Y = curr_frame["Ball"]["Y_coord"]
        Ball_H = curr_frame["Ball"]["Height"]
        Ball_W = curr_frame["Ball"]["Width"]
        
        if ((Ball_W*3 + LeBron_X) > Ball_X) and ((LeBron_X - Ball_W*3) < Ball_X):

            if ((Ball_H*3 + LeBron_Y) > Ball_Y) and ((LeBron_Y - Ball_H*3) < Ball_Y):

                prev_count = 1

        
    if prev_count != 0:
        prev_count += 1

    return prev_count

In [93]:
i = 0

wait_time = 60

waiting = 0
status = 0
FG = 0
FGA = 0

fourcc = cv2.VideoWriter_fourcc(*'MJPG')

cap = cv2.VideoCapture(r"C:/Users/GillA/Desktop/University/Cavs2003//Demo/demo3.mp4")

height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)

print(height)
was_bron_with_ball = 0

label_X = None
label_Y = None
label_time = 0

t = 0

writer = cv2.VideoWriter("Video/video_4/scorekeeping.avi", fourcc, 30, (int(width), int(height)))

ret, frame = cap.read()
while(1):
    ret, frame = cap.read()
    if cv2.waitKey(1) & 0xFF == ord('q') or ret==False :
        cap.release()
        cv2.destroyAllWindows()
        break
    
    i += 1

    
    curr_frame_info = frames_info["Frame_" + str(i)]
        
#     print(action)
    
    action = shot_check(curr_frame_info, waiting)

    font = cv2.FONT_HERSHEY_DUPLEX


    cv2.putText(frame, 
                "LeBron Career Field Goals: {}/{}".format(FG,FGA), 
                (270, 170), 
                font, 1.7, 
                (0, 255, 255), 
                6, 
                cv2.LINE_4)

    was_bron_with_ball = is_lebron_with_ball(was_bron_with_ball, curr_frame_info)


    if action or waiting > wait_time:


        
        if action == "Shot Up" and status == 0 and was_bron_with_ball != 0:
            status = 1
        
        print("Status {} Waiting {}".format(status,waiting))
        
        if status:
            waiting += 1
            
        if waiting > wait_time:
            FGA += 1
            print("Miss")
            status = 0
            waiting = 0
            
            label_Y = curr_frame_info["Net"]["Y_coord"] - curr_frame_info["Net"]["Height"]*0.5
            label_X = curr_frame_info["Net"]["X_coord"] - curr_frame_info["Net"]["Width"]*0.5
            label_type = "Miss"



            # messagebox.showinfo(title="Miss", message="LeBron Career Field Goals: {}/{}".format(FG,FGA))
#             cap.release()
#             cv2.destroyAllWindows()
#             break
            
        if action == "+1 FG":
            print(action)
            FG += 1
            FGA += 1
            label_Y = curr_frame_info["Net"]["Y_coord"] - curr_frame_info["Net"]["Height"]*0.5
            label_X = curr_frame_info["Net"]["X_coord"] - curr_frame_info["Net"]["Width"]*0.5
            label_type = "Make"
#             input("Hit Enter")
            
            # messagebox.showinfo(title="SCORE", message="LeBron Career Field Goals: {}/{}".format(FG,FGA))
#             cap.release()
#             cv2.destroyAllWindows()
#             break
            waiting = 0
            status = 0
            
#         input("Action Occurred")
            
    if status:
        waiting += 1

    if waiting > wait_time:
        status = 0
#         cap.release()
#         cv2.destroyAllWindows()
#         break
    
    if ("Net" in list(curr_frame_info.keys())) and label_X != None and label_Y != None:
        label_Y = curr_frame_info["Net"]["Y_coord"] - curr_frame_info["Net"]["Height"]*0.95
        label_X = curr_frame_info["Net"]["X_coord"] - curr_frame_info["Net"]["Width"]*0.5
        cv2.putText(frame, 
                    "{}".format(label_type), 
                    (int(label_X*width), int(label_Y*height)), 
                    font, 1, 
                    (255, 255, 255), 
                    4, 
                    cv2.LINE_4)

        label_time += 1

        shooting = label_type

        print("X : {} Y : {}".format(int(label_X*width),label_Y))
    
        if label_time > wait_time:
            label_X = None
            label_Y = None
            label_time = 0
    else:
            if waiting != 0:
                shooting = "Shot in Air"
            else:
                shooting = "No Shot"

        
    if was_bron_with_ball != 0:
        poss = "True"
    else:
        poss = "False"

    cv2.putText(frame, 
                "VARIABLES", 
                (1920-240, 150), 
                font, 0.9, 
                (255, 255, 255), 
                4, 
                cv2.LINE_4)

    cv2.putText(frame, 
                "Lebron has ball", 
                (1920-240, 200), 
                font, 0.9, 
                (0, 165, 255), 
                4, 
                cv2.LINE_4)
    
    cv2.putText(frame, 
            "{}".format(poss), 
            (1920-240, 250), 
            font, 0.9, 
            (0, 165, 255), 
            4, 
            cv2.LINE_4)
        
    # if (action == "+1 FG" or t > 0):
    #     t += 1
    #     shooting = label_type
    #     if t == 30:
    #         t = 0
    # elif waiting == 30:
    #     t += 1
    #     shooting = label_type
    #     if t == 30:
    #         t = 0


    cv2.putText(frame, 
                "Shot Status:", 
                (1920-240, 300), 
                font, 0.9, 
                (0, 0, 255), 
                4, 
                cv2.LINE_4)
    
    cv2.putText(frame, 
            "{}".format(shooting), 
            (1920-240, 350), 
            font, 0.9, 
            (0, 0, 255), 
            4, 
            cv2.LINE_4)
    
    time.sleep(0.01)
        
    writer.write(frame)
#     input("Press Enter to continue...")
    # cv2.imshow("frame",frame)
    
#     print(i)
        # cv2.imwrite("more\\2003Cavs_frame" + str(i) + ".jpg",frame)

writer.release()

1080.0
Status 1 Waiting 0
Status 1 Waiting 27
+1 FG
X : 883 Y : 0.27652745
X : 885 Y : 0.28662057500000004
X : 879 Y : 0.27870318000000005
X : 878 Y : 0.270046365
X : 879 Y : 0.260740635
X : 876 Y : 0.261759425
X : 871 Y : 0.25481509
X : 860 Y : 0.24314805999999997
X : 854 Y : 0.230138425
X : 848 Y : 0.223935605
X : 830 Y : 0.21319505000000002
X : 810 Y : 0.19643539999999998
X : 804 Y : 0.19324070000000004
X : 776 Y : 0.1943982
X : 757 Y : 0.1894451
X : 718 Y : 0.1772682
X : 699 Y : 0.1745828
X : 654 Y : 0.16537005000000002
X : 627 Y : 0.16310135
X : 598 Y : 0.16351805
X : 539 Y : 0.1477317
X : 514 Y : 0.14361100000000002
X : 478 Y : 0.13597150000000002
X : 409 Y : 0.12588004999999997
X : 372 Y : 0.11638855000000001
X : 329 Y : 0.10791665
X : 286 Y : 0.1024542
X : 252 Y : 0.0917126
X : 251 Y : 0.09060240000000003
X : 1112 Y : -0.04083326499999999
X : 1106 Y : -0.04833294999999999
X : 1103 Y : -0.04324105
X : 1100 Y : -0.027500350000000007
X : 1100 Y : -0.0010180499999999926
X : 1099 Y 

In [28]:
print("Shot Status: \n {}".format(shooting))

Shot Status: 
 No Shot


In [79]:
curr_frame_info["Net"] == None

KeyError: 'Net'

In [18]:
cap.release()
cv2.destroyAllWindows()
writer.release()

In [8]:
import sys

from PySide2.QtUiTools import QUiLoader
from PySide2.QtWidgets import QApplication, QMainWindow, QAction, QMessageBox, QFileDialog, QTextBrowser
from PySide2.QtCore import QFile


class MyMainWindow(QMainWindow):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setWindowTitle("Demo QtWidget App")

    def closeEvent(self, event):
        msgBox = QMessageBox()
        msgBox.setWindowTitle("Quit?")
        msgBox.setText("Exit application?")
        msgBox.setStandardButtons(QMessageBox.Yes | QMessageBox.No)
        msgBox.setDefaultButton(QMessageBox.No)
        ret = msgBox.exec_()
        if ret == QMessageBox.Yes:
            event.accept()
        else:
            event.ignore()


if __name__ == '__main__':
    app = QApplication([])
    file = QFile("custom_widget_original.ui")
    #file = QFile("custom_widget_modified.ui")
    file.open(QFile.ReadOnly)
    loader = QUiLoader()
    loader.registerCustomWidget(MyMainWindow)
    main_window = loader.load(file)
    main_window.show()
    sys.exit(app.exec_())


RuntimeError: Unable to open/read ui device

In [ ]:
curr_frame = frames_info["Frame_195"]
print(curr_frame)
list(curr_frame.keys())

In [ ]:
bool = 0

In [13]:
cv2.imshow("frame",frame)
cv2.destroyWindow("frame")

In [56]:
with open(r"C:\Users\Aseem Gill\Desktop\University\Cavs2003\Video\video_1\annots\mini_329.txt") as f:
    
    lines = f.readlines()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Aseem Gill\\Desktop\\University\\Cavs2003\\Video\\video_1\\annots\\mini_329.txt'

In [57]:
lines

['0 0.544792 0.617021 0.897917 0.744681\n']

In [38]:
classes = ["LeBron", "Ball", "Net"]

# Function for Getting Dicts
#### object-class x y width height

In [20]:
with open(r"C:\Users\Aseem Gill\Desktop\University\Cavs2003\Video\video_1\annots\mini_516.txt") as f:
    
    lines = f.readlines()

In [36]:
yolo_text_to_dict(lines, classes)

['0 0.913542 0.5 0.172917 1\n', '0 0.933333 0.513298 0.133333 0.904255\n', '0 0.528125 0.462766 0.19375 0.590426\n']
0 0.913542 0.5 0.172917 1

0 0.933333 0.513298 0.133333 0.904255

0 0.528125 0.462766 0.19375 0.590426



{'LeBron': {'X_coord': 0.528125,
  'Y_coord': 0.462766,
  'Width': 0.19375,
  'Height': 0.590426}}

In [34]:
print(frame_max)

901


In [9]:
help(model.predict)

Help on method predict in module ultralytics.yolo.engine.model:

predict(source=None, stream=False, **kwargs) method of ultralytics.yolo.engine.model.YOLO instance
    Perform prediction using the YOLO model.
    
    Args:
        source (str | int | PIL | np.ndarray): The source of the image to make predictions on.
                      Accepts all source types accepted by the YOLO model.
        stream (bool): Whether to stream the predictions or not. Defaults to False.
        **kwargs : Additional keyword arguments passed to the predictor.
                   Check the 'configuration' section in the documentation for all available options.
    
    Returns:
        (List[ultralytics.yolo.engine.results.Results]): The prediction results.



In [4]:
import cv2

In [7]:
i = 0

model = YOLO("Cavs2003_best.pt")

cap = cv2.VideoCapture(r"C:\Users\Aseem Gill\Desktop\University\LeBron James First NBA Game, Full Highlights vs Kings (2003.10.29) - MUST WATCH Debut! HD.mp4")
ret, frame = cap.read()
while(1):
    ret, frame = cap.read()
    if cv2.waitKey(1) & 0xFF == ord('q') or ret==False :
        cap.release()
        cv2.destroyAllWindows()
        break
    labelled = model.predict(frame,show=True)
    
    labelled = np.array(labelled)
    
    print(labelled)
    
    cv2.imshow("frame",labelled)
#     i += 1
#     if i in frames:
#         cv2.imwrite("images\\2003Cavs_frame" + str(i) + ".jpg",frame)

WARNING  Environment does not support cv2.imshow() or PIL Image.show()

Ultralytics YOLOv8.0.39  Python-3.9.12 torch-1.13.1+cpu CPU
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs

0: 384x640 143.6ms
Speed: 1.0ms pre-process, 143.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


[]


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
